In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import config
import utility.df
import utility.iolib
import utility.plotlib
from utility.feature import Feature

#env = sys.argv[1] if len(sys.argv) > 2 else "dev"

In [2]:
# Setup configuration
cfg = config.ResearchConfig
time_format = cfg.CSV_TIME_FORMAT
date_format = cfg.CSV_DATE_FORMAT
cutoff_date = pd.to_datetime(cfg.CUTOFF_DATE, format=cfg.CSV_DATE_FORMAT)

# Retrieve data
df_subspt, df_lesson, df_incomp, df_crclum, df_pupils = utility.iolib.retrieve_data(cfg)
print("Complete loading data for subscription and lesson history!")

# Filter data
cutoff_date = pd.to_datetime(cfg.CUTOFF_DATE, format=cfg.CSV_DATE_FORMAT)
first_date_impFromData = df_subspt.subscription_start_date.min()

pupils_toBeRemoved = utility.df.filter_subspt_data(
    df_subspt, first_date_impFromData, cutoff_date, remove_annual_subspt=False)
df_lesson1 = df_lesson[~df_lesson['pupilId'].isin(pupils_toBeRemoved)]
df_incomp1 = df_incomp[~df_incomp['pupilId'].isin(pupils_toBeRemoved)]
df_subspt1 = df_subspt[~df_subspt['pupilId'].isin(pupils_toBeRemoved)]

df_subspt1 = utility.df.compute_customer_month(df_subspt1, cfg)

Complete loading data for subscription and lesson history!
By the cutoff date 2018-04-20, there are 1234 active subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to know the lifetime of their subscriptions. 

In the first month of dataset starting from 2014-01-01, there are 154 renewal or new subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to show if they renewed or newly joined. 

In summary, there are 1380/5685 subscribers being removed from the dataset in the analysis. 

Calculate customer month in the subscription table.


100%|██████████| 4305/4305 [00:06<00:00, 662.27it/s] 


In [21]:
# outcome

mask_fwrd = (df_lesson1['outcome']=='p') & (df_lesson1['run_mode']=='j')
num_fwrd = df_lesson1[mask_fwrd].groupby(['pupilId', 'date'])['outcome'].count()

mask_pass = ((df_lesson1['outcome']=='p') & (df_lesson1['run_mode']!='j')) |\
    ((df_lesson1['outcome']=='s') & (df_lesson1['run_mode']=='j'))
num_pass = df_lesson1[mask_pass].groupby(['pupilId', 'date'])['outcome'].count()

mask_stat = (df_lesson1['outcome']=='s') & (df_lesson1['run_mode']!='j')
num_stat = df_lesson1[mask_stat].groupby(['pupilId', 'date'])['outcome'].count()

mask_fail = (df_lesson1['outcome']=='f') & (df_lesson1['run_mode']=='x')
num_fail = df_lesson1[mask_fail].groupby(['pupilId', 'date'])['outcome'].count()

mask_back = (df_lesson1['outcome']=='f') & (df_lesson1['run_mode']=='b')
num_back = df_lesson1[mask_back].groupby(['pupilId', 'date'])['outcome'].count()

mask_repl = (df_lesson1['outcome']=='0') & (df_lesson1['run_mode']=='r')
num_repl = df_lesson1[mask_repl].groupby(['pupilId', 'date'])['outcome'].count()

num_fwrd.rename('num_fwrd', inplace=True)
num_pass.rename('num_pass', inplace=True)
num_stat.rename('num_stat', inplace=True)
num_fail.rename('num_fail', inplace=True)
num_back.rename('num_back', inplace=True)
num_repl.rename('num_repl', inplace=True)

num_outcome = pd.concat([num_fwrd, num_pass, num_stat, num_fail, num_back, num_repl], axis=1)
num_outcome.fillna(0, inplace=True)

In [29]:
num_outcome['num_attempt'] = num_outcome.sum(axis=1)
num_outcome['num_assess'] = num_outcome['num_attempt'] - num_outcome['num_repl']

In [30]:
num_outcome

num_fwrd  num_pass  num_stat  num_fail  num_back  \
pupilId date                                                           
655886  2014-04-18       0.0       2.0       0.0       0.0       0.0   
        2014-04-19       0.0       1.0       0.0       0.0       0.0   
        2014-04-20       0.0       0.0       2.0       0.0       0.0   
        2014-04-21       0.0       2.0       1.0       0.0       0.0   
        2014-05-18       0.0       3.0       2.0       0.0       0.0   
        2014-10-21       0.0       6.0       3.0       0.0       0.0   
        2014-10-22       0.0       2.0       1.0       0.0       0.0   
        2014-12-23       0.0       8.0       1.0       0.0       0.0   
        2015-03-06       0.0       3.0       0.0       0.0       0.0   
        2015-05-17       1.0       5.0       1.0       0.0       0.0   
658323  2013-12-12       0.0       1.0       0.0       0.0       0.0   
        2013-12-27       0.0       2.0       0.0       0.0       0.0   
        2014-02-23       0.0       1.0       0.0       0.0       0.0   
        2016-03-10       1.0       8.0       0.0       0.0       0.0   
        2016-03-11       0.0       1.0       0.0       0.0       0.0   
        2016-03-13      10.0      17.0       0.0       0.0       0.0   
        2016-03-14       1.0       1.0       0.0       0.0       0.0   
        2016-03-16       0.0       5.0       0.0       0.0       0.0   
        2016-03-17       2.0       2.0       1.0       0.0       0.0   
        2016-03-19       2.0       3.0       0.0       0.0       0.0   
        2016-03-21       2.0       4.0       0.0       0.0       0.0   
        2016-03-22       4.0       6.0       1.0       0.0       0.0   
        2016-03-23       2.0       3.0       0.0       0.0       0.0   
        2016-03-25       0.0       0.0       0.0       0.0       0.0   
        2016-03-29       2.0       9.0       1.0       0.0       0.0   
        2016-04-12       0.0       4.0       0.0       0.0       0.0   
        2016-04-13       3.0      10.0       1.0       0.0       0.0   
        2016-04-16       2.0       3.0       0.0       0.0       0.0   
        2016-04-19       3.0       3.0       0.0       0.0       0.0   
        2016-04-20       1.0       4.0       1.0       0.0       0.0   
...                      ...       ...       ...       ...       ...   
1076541 2016-09-18       1.0       2.0       0.0       0.0       0.0   
        2016-09-20       1.0       1.0       0.0       0.0       0.0   
        2016-09-25       2.0       2.0       0.0       0.0       0.0   
        2016-10-02       2.0       4.0       0.0       0.0       0.0   
        2016-10-04       1.0       3.0       0.0       0.0       0.0   
        2016-10-15       1.0       5.0       0.0       0.0       0.0   
        2016-10-19       0.0       2.0       0.0       0.0       0.0   
        2016-10-22       0.0       4.0       1.0       0.0       0.0   
        2016-10-23       3.0       3.0       0.0       0.0       0.0   
        2016-11-05       1.0       7.0       0.0       0.0       0.0   
        2016-11-06       0.0       4.0       0.0       0.0       0.0   
        2016-11-10       1.0       1.0       0.0       0.0       0.0   
        2016-11-13       0.0       4.0       0.0       0.0       0.0   
        2016-11-15       1.0       1.0       0.0       0.0       0.0   
        2016-11-19       1.0       5.0       0.0       0.0       0.0   
        2016-11-22       0.0       3.0       0.0       0.0       0.0   
        2016-12-18       0.0       5.0       0.0       0.0       0.0   
        2016-12-20       1.0       1.0       0.0       0.0       0.0   
        2016-12-21       1.0       1.0       0.0       0.0       0.0   
        2016-12-22       0.0       2.0       0.0       0.0       0.0   
        2016-12-23       0.0       2.0       0.0       0.0       0.0   
        2017-01-01       0.0       2.0       0.0       0.0       0.0   
        2017-01-02       1.0       5.0       0.0       0.0  